In [2]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [7]:
small_model_name = "ai-forever/rugpt3small_based_on_gpt2"

tokenizer_small = AutoTokenizer.from_pretrained(small_model_name)
model_small = AutoModelForCausalLM.from_pretrained(small_model_name)
tokenizer_small.pad_token_id = tokenizer_small.eos_token_id

small_params = count_parameters(model_small)

print(f"=== {small_model_name} ===")
print(f"Общее число параметров: {small_params}")
print("Архитектура модели (слои и их размеры):")
print(model_small)

=== ai-forever/rugpt3small_based_on_gpt2 ===
Общее число параметров: 125231616
Архитектура модели (слои и их размеры):
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm

In [9]:
large_model_name = "ai-forever/rugpt3large_based_on_gpt2"

tokenizer_large = AutoTokenizer.from_pretrained(large_model_name)
model_large = AutoModelForCausalLM.from_pretrained(large_model_name)
tokenizer_large.pad_token_id = tokenizer_large.eos_token_id

large_params = count_parameters(model_large)

print(f"=== {large_model_name} ===")
print(f"Общее число параметров: {large_params}")
print("Архитектура модели (слои и их размеры):")
model_large

=== ai-forever/rugpt3large_based_on_gpt2 ===
Общее число параметров: 760300032
Архитектура модели (слои и их размеры):


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=4608, nx=1536)
          (c_proj): Conv1D(nf=1536, nx=1536)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=6144, nx=1536)
          (c_proj): Conv1D(nf=1536, nx=6144)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1536, out_features=50257, bias=False)
)

In [12]:
prompts = [
    "Расскажи подробно, что такое нутрициология?",
    "Как правильно составить рацион питания для поддержания здоровья?",
    "Какая роль белков, жиров и углеводов в организме человека?",
    "Как нутрициология влияет на общее состояние здоровья?",
    "Как оптимизировать питание для спортсменов и повышения выносливости?",
    "Подскажи мне 5 блюд пищи, если я легкоатлет и придерживаюсь спортивной диеты"
]

In [15]:
models = {
    "ruGPT3Small": (tokenizer_small, model_small),
    "ruGPT3Large": (tokenizer_large, model_large)
}

In [ ]:
for model_label, (tokenizer, model) in models.items():
    print(f"\n{'='*10} Модель: {model_label} {'='*10}\n")
    model.eval()
    model.to("cpu")
    
    for prompt in prompts:
        start_time = time.time()
        
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(
            **inputs,
            max_length=128,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.1,
            no_repeat_ngram_size=3,
        )
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # answer = tokenizer.decode(outputs[0]).split("<s>")[0]
        
        end_time = time.time()
        generation_time = end_time - start_time
        
        print("Промпт:", prompt)
        print("Ответ модели:", answer)
        print("Время генерации на CPU: {:.2f} секунд".format(generation_time))
        print("-" * 60)


========== Модель: ruGPT3Small ==========

Промпт: Расскажи подробно, что такое нутрициология?
Ответ модели: Расскажи подробно, что такое нутрициология?
Нутриционология - это наука о составе и функциях пищи.
Врач, диетолог, врач-эндокринолог...

Что делать если в скайпе не могу войти в настройки скайпа?
А ты уверен, что в Скайпе нет такого режима?

Как понять кто врёт?
Если вы не доверяете человеку, то он вам не поверит. Вы сами обманываетесь.
Проверьте его слова
это из за него вы не сможете зайти на сайт
Если Вы не уверены в правдивости его слов
Время генерации на CPU: 1.44 секунд
------------------------------------------------------------
Промпт: Как правильно составить рацион питания для поддержания здоровья?
Ответ модели: Как правильно составить рацион питания для поддержания здоровья?
Это очень важный вопрос. И ответ на него в этой статье будет один - не есть после шести часов.
Питание для организма - это как и что есть, так и как есть. Не кушать за столом и не пить чай с молоко